## Step 1: Hello, Data!
Load raw CSV, display first 3 rows

In [1]:
import pandas as pd
df = pd.read_csv('data/1000SalesRecords.csv')
df.head(3)

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Middle East and North Africa,Libya,Cosmetics,Offline,M,10/18/2014,686800706,10/31/2014,8446,437.20,263.33,3692591.20,2224085.18,1468506.02
1,North America,Canada,Vegetables,Online,M,11/7/2011,185941302,12/8/2011,3018,154.06,90.93,464953.08,274426.74,190526.34
2,Middle East and North Africa,Libya,Baby Food,Offline,C,10/31/2016,246222341,12/9/2016,1517,255.28,159.42,387259.76,241840.14,145419.62


## Step 2: Pick the Right Container
Dictionaries are more flexible for mapping keys (like Order ID ➞ record). Namedtuples are lightweight but less flexible for transformations. Sets are best when we only need uniqueness (e.g., unique cities).

## Step 3: Implement Functions and Data Structure

In [2]:
def build_order_dict(row):
    return {
        'order_id': row['Order ID'],
        'country': row['Country'],
        'item': row['Item Type'],
        'units_sold': row['Units Sold'],
        'total_revenue': row['Total Revenue']
    }

order_dicts = df.head(5).apply(build_order_dict, axis=1).tolist()
order_dicts

[{'order_id': 686800706,
  'country': 'Libya',
  'item': 'Cosmetics',
  'units_sold': 8446,
  'total_revenue': 3692591.2},
 {'order_id': 185941302,
  'country': 'Canada',
  'item': 'Vegetables',
  'units_sold': 3018,
  'total_revenue': 464953.08},
 {'order_id': 246222341,
  'country': 'Libya',
  'item': 'Baby Food',
  'units_sold': 1517,
  'total_revenue': 387259.76},
 {'order_id': 161442649,
  'country': 'Japan',
  'item': 'Cereal',
  'units_sold': 3322,
  'total_revenue': 683335.4},
 {'order_id': 645713555,
  'country': 'Chad',
  'item': 'Fruits',
  'units_sold': 9845,
  'total_revenue': 91853.85}]

## Step 4: Bulk Loaded
Map DataFrame rows into dictionaries

In [3]:
orders = df.apply(build_order_dict, axis=1).tolist()
len(orders)

1000

## Step 5: Quick Profiling

In [4]:
min_price = df['Unit Price'].min()
max_price = df['Unit Price'].max()
mean_price = df['Unit Price'].mean()
unique_cities = len(set(df['Country']))
min_price, mean_price, max_price, unique_cities

(np.float64(9.33), np.float64(262.10684), np.float64(668.27), 185)

## Step 6: Spot the Grime
- Missing or blank values
- Inconsistent date formats
- Negative or zero units sold


## Step 7: Cleaning Rules

In [5]:
def clean(df):
    before = len(df)
    df = df.dropna()
    df = df[df['Units Sold'] > 0]
    after = len(df)
    return df, before, after

df_clean, before, after = clean(df)
before, after

(1000, 1000)

## Step 8: Transformations

In [6]:
# Example: Suppose coupon_code column exists
import numpy as np
if 'coupon_code' not in df_clean.columns:
    df_clean['coupon_code'] = np.where(df_clean['Order Priority']=='H','DISC10','NONE')

def parse_coupon(code):
    if code.startswith('DISC'):
        return int(code.replace('DISC',''))
    return 0

df_clean['discount_percent'] = df_clean['coupon_code'].apply(parse_coupon)
df_clean[['coupon_code','discount_percent']].head()

,coupon_code,discount_percent
0,NONE,0
1,NONE,0
2,NONE,0
3,NONE,0
4,DISC10,10


## Step 9: Feature Engineering

In [7]:
df_clean['Order Date'] = pd.to_datetime(df_clean['Order Date'])
df_clean['Ship Date'] = pd.to_datetime(df_clean['Ship Date'])
df_clean['days_since_purchase'] = (df_clean['Ship Date'] - df_clean['Order Date']).dt.days
df_clean[['Order ID','days_since_purchase']].head()

,Order ID,days_since_purchase
0,686800706,13
1,185941302,31
2,246222341,39
3,161442649,32
4,645713555,15


## Step 10: Mini-Aggregation

In [8]:
revenue_per_country = df_clean.groupby('Country')['Total Revenue'].sum().to_dict()
list(revenue_per_country.items())[:5]

[('Afghanistan', 2843589.07),
 ('Albania', 9709899.27),
 ('Algeria', 10272591.440000001),
 ('Andorra', 7153122.97),
 ('Angola', 15643032.02)]

## Step 11: Serialization Checkpoint

In [9]:
df_clean.to_json('data/cleaned_sales.json', orient='records', lines=True)
print('Saved to data/cleaned_sales.json')

Saved to data/cleaned_sales.json


## Step 12: Soft Interview Reflection
Functions helped modularize the workflow: cleaning, transforming, and mapping steps became reusable units instead of repeated code. Using a function for building dictionaries and another for cleaning ensured clarity, reduced duplication, and made the code easier to debug and extend.

## Data-Dictionary Section

| Field | Type | Description | Source |
|-------|------|-------------|--------|
| Region | str | Sales region | CSV |
| Country | str | Country of order | CSV |
| Item Type | str | Product category | CSV |
| Sales Channel | str | Online/Offline channel | CSV |
| Order Priority | str | Priority level | CSV |
| Order Date | date | Purchase date | CSV |
| Order ID | int | Unique order identifier | CSV |
| Ship Date | date | Shipping date | CSV |
| Units Sold | int | Quantity sold | CSV |
| Unit Price | float | Price per unit | CSV |
| Unit Cost | float | Cost per unit | CSV |
| Total Revenue | float | Units Sold × Unit Price | CSV |
| Total Cost | float | Units Sold × Unit Cost | CSV |
| Total Profit | float | Revenue – Cost | CSV |
| coupon_code | str | Synthetic field from priority | Synthetic |
| discount_percent | int | Parsed from coupon_code | Derived |
| days_since_purchase | int | Difference between Ship Date and Order Date | Derived |